# Test time-series tool for QGIS

In [12]:
import ee

# The service account email address authorized by your Google contact.
# Set up a service account as described in the README.
EE_ACCOUNT = "gef-ldmp-server@gef-ld-toolbox.iam.gserviceaccount.com"

# The private key associated with your service account in JSON format.
EE_PRIVATE_KEY_FILE = "D:/Code/LandDegradation/decisiontheater/te_key.json"

EE_CREDENTIALS = ee.ServiceAccountCredentials(EE_ACCOUNT, EE_PRIVATE_KEY_FILE)
ee.Initialize(EE_CREDENTIALS)
ee.Initialize()

In [13]:
import json
import logging
import random
import re
from builtins import str, zip

from te_algorithms.gee.productivity import productivity_series
from te_schemas.schemas import TimeSeries, TimeSeriesTable, TimeSeriesTableSchema

logger = logging.getLogger(__name__)

In [14]:
params = {
    "climate_gee_dataset": "users/geflanddegradation/toolbox_datasets/prec_chirps_1981_2021",
    "crosses_180th": False,
    "crs": 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]',
    "geojsons": '[{"coordinates": [[[89.73584598190655, 26.696149394000116], [90.95783857638924, 26.696149394000116], [90.95783857638924, 27.20932139716476], [89.73584598190655, 27.20932139716476], [89.73584598190655, 26.696149394000116]]], "type": "Polygon"}]',
    "ndvi_gee_dataset": "users/geflanddegradation/toolbox_datasets/ndvi_modis_2001_2021",
    "trajectory_method": "p_restrend",
    "year_final": 2021,
    "year_initial": 2001,
}

In [42]:
def zonal_stats(
    geojsons,
    year_initial,
    year_final,
    trajectory_method,
    ndvi_gee_dataset,
    climate_gee_dataset,
    logger,
):
    logger.debug("Entering zonal_stats function.")

    image = (
        productivity_series(
            int(year_initial),
            int(year_final),
            trajectory_method,
            ndvi_gee_dataset,
            climate_gee_dataset,
            logger,
        )
        .select("ndvi")
        .toBands()
    )

    region = ee.Geometry(geojsons)

    scale = ee.Number(image.projection().nominalScale()).getInfo()

    ## This produces an average of the region over the image by year
    ## Source: https://developers.google.com/earth-engine/reducers_reduce_region
    reducers = (
        ee.Reducer.mean()
        .combine(reducer2=ee.Reducer.min(), sharedInputs=True)
        .combine(reducer2=ee.Reducer.max(), sharedInputs=True)
        .combine(reducer2=ee.Reducer.mode(), sharedInputs=True)
        .combine(reducer2=ee.Reducer.stdDev(), sharedInputs=True)
    )
    statsDictionary = image.reduceRegion(
        reducer=reducers, geometry=region, scale=scale, maxPixels=1e13
    )

    logger.debug("Calculating zonal_stats.")
    res = statsDictionary.getInfo()

    logger.debug("Formatting results.")
    res_clean = {}

    years = [*range(year_initial, year_final + 1)]
    for key, value in list(res.items()):
        re_groups = re.search("(\d*)_(\w*)", key).groups()
        index = re_groups[0]
        year = years[int(index)]
        field = re_groups[1]
        
        if field not in res_clean:
            res_clean[field] = {}
            res_clean[field]["value"] = []
            res_clean[field]["year"] = []
        res_clean[field]["value"].append(float(value))
        res_clean[field]["year"].append(int(year))

    logger.debug("Setting up results JSON")
    timeseries = []

    for key in list(res_clean.keys()):
        # Ensure the lists are in chronological order
        year, value = list(
            zip(*sorted(zip(res_clean[key]["year"], res_clean[key]["value"])))
        )
        ts = TimeSeries(list(year), list(value), key)
        timeseries.append(ts)

    timeseries_table = TimeSeriesTable("timeseries", timeseries)
    timeseries_table_schema = TimeSeriesTableSchema()
    json_result = timeseries_table_schema.dump(timeseries_table)

    return json_result

In [41]:
image = (
    productivity_series(
        int(year_initial),
        int(year_final),
        trajectory_method,
        ndvi_gee_dataset,
        climate_gee_dataset,
        logger,
    )
    .select("ndvi")
    .toBands()
)

region = ee.Geometry(geojsons[0])

scale = ee.Number(image.projection().nominalScale()).getInfo()

## This produces an average of the region over the image by year
## Source: https://developers.google.com/earth-engine/reducers_reduce_region
reducers = (
    ee.Reducer.mean()
    .combine(reducer2=ee.Reducer.min(), sharedInputs=True)
    .combine(reducer2=ee.Reducer.max(), sharedInputs=True)
    .combine(reducer2=ee.Reducer.mode(), sharedInputs=True)
    .combine(reducer2=ee.Reducer.stdDev(), sharedInputs=True)
)
statsDictionary = image.reduceRegion(
    reducer=reducers, geometry=region, scale=scale, maxPixels=1e13
)

logger.debug("Calculating zonal_stats.")
res = statsDictionary.getInfo()

logger.debug("Formatting results.")
res_clean = {}

years = [*range(year_initial, year_final + 1)]
for key, value in list(res.items()):
    re_groups = re.search("(\d*)_(\w*)", key).groups()
    index = re_groups[0]
    year = years[int(index)]
    field = re_groups[1]
    

    if field not in res_clean:
        res_clean[field] = {}
        res_clean[field]["value"] = []
        res_clean[field]["year"] = []
    res_clean[field]["value"].append(float(value))
    res_clean[field]["year"].append(int(year))

print(res_clean)

{'ndvi_max': {'value': [-20.298712891539253, 69.75337084635794, 236.86592895579543, 31.309200901736403, 136.49544774126616, 626.0471677512078, 153.92202844297663, 540.6781601376028, 570.3179114784771, 631.2566358577483, 489.1332327970704, -49.84760867764999, 430.31326066037946, -142.94793314166145, -109.18966259438457, -3.420149071400374, -58.15354447984464, 103.67251847355419, -38.21928456404021, 120.44181660916729, 24.20228103633235], 'year': [2001, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2002, 2021, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]}, 'ndvi_mean': {'value': [-419.83891694877707, -64.96720254065022, 105.68896813631554, -202.0944013612229, 45.98613771587721, 378.1512024981405, 99.12355714249806, 367.459162277705, 350.4101365387817, 414.3876350537403, 295.47827431915925, -115.24624374625806, 187.29244171326425, -338.08701543914106, -327.76351891293075, -235.3787097780539, -120.9360827329493, -16.873152793465803, -296.4657966613747, 21.315652258027427, 

In [43]:
logger.debug("Loading parameters.")
geojsons = json.loads(params.get("geojsons"))
year_initial = int(params.get("year_initial"))
year_final = int(params.get("year_final"))
trajectory_method = params.get("trajectory_method")
ndvi_gee_dataset = params.get("ndvi_gee_dataset")
climate_gee_dataset = params.get("climate_gee_dataset")

# Check the ENV. Are we running this locally or in prod?

if params.get("ENV") == "dev":
    EXECUTION_ID = str(random.randint(1000000, 99999999))
else:
    EXECUTION_ID = params.get("EXECUTION_ID", None)
logger.debug(f"Execution ID is {EXECUTION_ID}")

logger.debug("Running main script.")
# TODO: Right now timeseries will only work on the first geojson - this is
# somewhat ok since for the most part this uses points, but should fix in
# the future
json_result = zonal_stats(
    geojsons[0],
    year_initial,
    year_final,
    trajectory_method,
    ndvi_gee_dataset,
    climate_gee_dataset,
    logger,
)
print(json_result)

{'table': [{'time': [2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0, 2021.0], 'name': 'ndvi_max', 'y': [-20.298712891539253, -49.84760867764999, -142.94793314166145, -109.18966259438457, -3.420149071400374, -58.15354447984464, 103.67251847355419, -38.21928456404021, 120.44181660916729, 24.20228103633235, 69.75337084635794, 236.86592895579543, 31.309200901736403, 136.49544774126616, 626.0471677512078, 153.92202844297663, 540.6781601376028, 570.3179114784771, 631.2566358577483, 489.1332327970704, 430.31326066037946]}, {'time': [2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0, 2021.0], 'name': 'ndvi_mean', 'y': [-419.83891694877707, -115.24624374625806, -338.08701543914106, -327.76351891293075, -235.3787097780539, -120.9360827329493, -16.873152793465803, -296.4657966613